# Clasificación para prescripción de medicamentos

En el dataset "drug200.csv" se encuentran los datos de medicamentos recetados para una determinada patología en función de las características de cada paciente.
- Separa en conjuntos de entrenamiento y test garantizando que haya muestras de todos los medicamentos en ambos conjuntos.
- Entrena un modelo que prediga el medicamento que se le recetará a un paciente en función de sus características.
- Crea un pipeline con los pasos seguidos.

In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline

df_drug = pd.read_csv("data/drug200.csv")

In [96]:
print(df_drug.head())

   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  DrugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  DrugY


In [97]:
print(df_drug.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB
None


In [98]:
print(df_drug.describe())

              Age     Na_to_K
count  200.000000  200.000000
mean    44.315000   16.084485
std     16.544315    7.223956
min     15.000000    6.269000
25%     31.000000   10.445500
50%     45.000000   13.936500
75%     58.000000   19.380000
max     74.000000   38.247000


In [99]:
print(df_drug.isnull().sum())

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64


## Separación de datos

In [100]:
X = df_drug.drop(columns=["Drug"])
y = df_drug["Drug"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

## Procesado de columnas categóricas

In [101]:
print(X_train['Sex'].value_counts())
print(X_train['BP'].value_counts())
print(X_train['Cholesterol'].value_counts())

Sex
M    85
F    75
Name: count, dtype: int64
BP
HIGH      63
LOW       50
NORMAL    47
Name: count, dtype: int64
Cholesterol
NORMAL    82
HIGH      78
Name: count, dtype: int64


In [108]:
X_train_tr = X_train.copy()

X_train_tr['Sex'] = X_train['Sex'].map({'M': 0, 'F': 1})
X_train_tr['BP'] = X_train['BP'].map({'LOW': 0, 'NORMAL': 1, 'HIGH': 2}) # Importante el orden
X_train_tr['Cholesterol'] = X_train['Cholesterol'].map({'NORMAL': 0, 'HIGH': 1})

Age  Sex  BP  Cholesterol  Na_to_K
52    62    0   0            0   27.183
34    53    0   1            1   14.133
113   65    1   0            0   13.769
168   51    1   0            0   23.003
127   35    0   1            0    7.845
..   ...  ...  ..          ...      ...
30    18    1   1            0    8.750
142   60    0   2            0    8.621
196   16    0   0            1   12.006
170   28    1   1            1   12.879
199   40    1   0            0   11.349

[160 rows x 5 columns]

## Modelo con regresión logística:

In [103]:
log_reg = LogisticRegression(max_iter=10000).fit(X_train_tr, y_train)

## Modelo con árbol de decisión:

In [104]:
tree = DecisionTreeClassifier().fit(X_train_tr,y_train)

## Exactitud de los modelos:

Aunque no se pedía, es interesante ver cómo se podría testear la exactitud del modelo. Es importante hacer el mismo preprocesado que en el conjunto de entrenamiento. Para esto es útil el pipeline; para no tener que repetir ese código, exponiéndose a errores. Con el pipeline tendremos un objeto que representará todo el procesamiento de los datos, y podremos aplicarlo a cualquier conjunto de datos.

In [105]:
X_test_tr = X_test.copy()
X_test_tr['Sex'] = X_test['Sex'].map({'M': 0, 'F': 1})
X_test_tr['BP'] = X_test['BP'].map({'LOW': 0, 'NORMAL': 1, 'HIGH': 2}) # Importante el orden
X_test_tr['Cholesterol'] = X_test['Cholesterol'].map({'NORMAL': 0, 'HIGH': 1})


y_pred_log_reg = log_reg.predict(X_test_tr)
y_pred_tree = tree.predict(X_test_tr)

print("Exactitud con función 'accuracy_score': ", accuracy_score(y_test, y_pred_log_reg))
print("Exactitud con método 'score': ", log_reg.score(X_test_tr, y_test))
print("Exactitud con cálculo manual: ", np.sum(y_pred_log_reg==y_test)/len(y_test))

print("Exactitud con función 'accuracy_score': ", accuracy_score(y_test, y_pred_tree))
print("Exactitud con método 'score': ", log_reg.score(X_test_tr, y_test))
print("Exactitud con cálculo manual: ", np.sum(y_pred_tree==y_test)/len(y_test))

Exactitud con función 'accuracy_score':  0.975
Exactitud con método 'score':  0.975
Exactitud con cálculo manual:  0.975
Exactitud con función 'accuracy_score':  0.975
Exactitud con método 'score':  0.975
Exactitud con cálculo manual:  0.975


## Pipeline

In [106]:
ordinal_transformer = ColumnTransformer(
    transformers=[
        ('sex', OrdinalEncoder(), ['Sex']),
        ('bp', OrdinalEncoder(categories=[['LOW', 'NORMAL', 'HIGH']]), ['BP']),
        ('cholesterol', OrdinalEncoder(), ['Cholesterol'])
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(ordinal_transformer, DecisionTreeClassifier()) 

model = pipeline.fit(X_train, y_train)

model


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex', OrdinalEncoder(),
                                                  ['Sex']),
                                                 ('bp',
                                                  OrdinalEncoder(categories=[['LOW',
                                                                              'NORMAL',
                                                                              'HIGH']]),
                                                  ['BP']),
                                                 ('cholesterol',
                                                  OrdinalEncoder(),
                                                  ['Cholesterol'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [107]:
model.score(X_test, y_test)

0.975